In [1]:
import re
import pandas as pd

In [2]:
required_genomes = [
    "ON107264", "MN270259", "NC_030940", "MK448731",
    "KY695241", "NC_031039", "NC_043027", "MW749003", "NC_007581", "MZ422438", 
    "MW248466", "LC680885", "MN091626", "OQ555808", "EU982300", 
    "ON470608", "MK448228", "MZ333456", "HQ906664"
]

## Read in the hmmsearch results and filter

In [3]:
# Function to read HMMER .tblout file into a pandas DataFrame
def read_tblout_to_dataframe(tblout_file):
    # Define column names based on HMMER .tblout format
    column_names = [
        'target_name', 'accession', 'query_name', 'accession_query', 
        'e_value', 'score', 'bias', 'e_value_best_dom', 
        'score_best_dom', 'bias_best_dom', 'exp', 'reg', 
        'clu', 'ov', 'env', 'dom', 'rep', 'inc', 'description'
    ]
    
    # Read the file, skipping lines starting with '#'
    data = []
    with open(tblout_file, 'r') as f:
        for line in f:
            if not line.startswith("#"):  # Skip comment lines
                # Split the line by any whitespace and limit to the first 18 fields
                fields = line.strip().split(maxsplit=18)
                data.append(fields)

    # Create a pandas DataFrame
    df = pd.DataFrame(data, columns=column_names)

    df['genome'] = df['target_name'].str.split('_00').str[0]

    # Convert the 'e_value' column to float for numerical filtering
    df['e_value'] = pd.to_numeric(df['e_value'], errors='coerce')
        
    # Step 1: Identify duplicated genomes
    duplicated_genomes = df[df['genome'].duplicated(keep=False)]

    # Step 2: For duplicated genomes, keep only the row with the lowest 'e_value'
    lowest_evalue_duplicates = duplicated_genomes.loc[duplicated_genomes.groupby('genome')['e_value'].idxmin()]

    # Step 3: Get the rows where genome is not duplicated (keep as they are)
    unique_genomes = df[~df['genome'].duplicated(keep=False)]

    # Step 4: Combine both unique genomes and lowest e_value duplicates
    final_df = pd.concat([lowest_evalue_duplicates, unique_genomes], ignore_index=True)

    return final_df.loc[final_df['e_value'] <= 1]

tblout_file = './PF03237_millard_sept_2.tblout'
df = read_tblout_to_dataframe(tblout_file)

# Filter the DataFrame for e-values <= 0.1
filtered_df = df[(df['e_value'] <= 0.01) | (df.genome.isin(required_genomes))]
filtered_df

,target_name,accession,query_name,accession_query,e_value,score,bias,e_value_best_dom,score_best_dom,bias_best_dom,exp,reg,clu,ov,env,dom,rep,inc,description,genome
0,AB472900_00159,-,Terminase_6N,PF03237.20,7.200000e-32,117.7,0.0,1.4e-31,116.8,0.0,1.4,2,0,0,2,2,1,1,terminase large subunit,AB472900
1,AB797215_00147,-,Terminase_6N,PF03237.20,9.800000e-41,146.7,0.2,1.9e-40,145.8,0.2,1.4,1,0,0,1,1,1,1,terminase large subunit,AB797215
2,AB897757_00112,-,Terminase_6N,PF03237.20,5.300000e-33,121.4,0.5,5.3e-33,121.4,0.5,2.2,2,0,0,2,2,1,1,terminase large subunit,AB897757
3,AC171169_00136,-,Terminase_6N,PF03237.20,5.700000e-46,163.8,0.2,1.3e-45,162.7,0.1,1.5,2,0,0,2,2,1,1,terminase large subunit,AC171169
6,AF158101_00166,-,Terminase_6N,PF03237.20,8.700000e-69,238.4,3.8,3.7e-68,236.3,0.1,2.2,2,0,0,2,2,1,1,terminase large subunit,AF158101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13821,KX620752_00002,-,Terminase_6N,PF03237.20,1.000000e-02,22.4,0.0,0.05,20.2,0.1,2.0,2,1,0,2,2,1,0,terminase large subunit,KX620752
13822,KY087898_00048,-,Terminase_6N,PF03237.20,1.000000e-02,22.4,0.0,0.026,21.1,0.0,1.7,1,1,0,1,1,1,0,terminase large subunit,KY087898
13823,MN812688_00048,-,Terminase_6N,PF03237.20,1.000000e-02,22.4,0.0,0.026,21.1,0.0,1.7,1,1,0,1,1,1,0,terminase large subunit,MN812688
14699,MW749003_00001,-,Terminase_6N,PF03237.20,1.200000e-01,18.9,0.0,0.19,18.3,0.0,1.3,1,0,0,1,1,1,0,terminase large subunit,MW749003


#### Check how many of our phages got hit with the pfam terminase model

In [4]:
filtered_df.loc[filtered_df.genome.isin(required_genomes)]

,target_name,accession,query_name,accession_query,e_value,score,bias,e_value_best_dom,score_best_dom,bias_best_dom,exp,reg,clu,ov,env,dom,rep,inc,description,genome
2262,ON107264_00182,-,Terminase_6N,PF03237.20,8.000000e-15,62.0,0.9,2.7e-14,60.3,0.5,2.2,2,1,0,2,2,1,1,hypothetical protein,ON107264
6136,OQ555808_00008,-,Terminase_6N,PF03237.20,4.900000e-22,85.6,0.1,7.2e-22,85.0,0.1,1.2,1,0,0,1,1,1,1,unannotated protein,OQ555808
7423,MZ333456_00018,-,Terminase_6N,PF03237.20,1.100000e-17,71.3,0.2,1.9e-17,70.6,0.2,1.3,1,0,0,1,1,1,1,terminase large subunit,MZ333456
10283,LC680885_00014,-,Terminase_6N,PF03237.20,1.500000e-07,38.2,1.7,3.9e-07,36.9,1.7,1.8,1,1,0,1,1,1,1,terminase large subunit,LC680885
10287,MN091626_00150,-,Terminase_6N,PF03237.20,1.500000e-07,38.2,0.0,2.2e-07,37.7,0.0,1.3,1,0,0,1,1,1,1,terminase large subunit,MN091626
10292,MN270259_00030,-,Terminase_6N,PF03237.20,1.500000e-07,38.2,0.0,2.8e-07,37.3,0.0,1.4,1,0,0,1,1,1,1,terminase large subunit,MN270259
11191,ON470608_00029,-,Terminase_6N,PF03237.20,1.500000e-05,31.7,0.0,3.1e-05,30.7,0.0,1.6,1,1,0,1,1,1,1,terminase large subunit,ON470608
14699,MW749003_00001,-,Terminase_6N,PF03237.20,1.200000e-01,18.9,0.0,0.19,18.3,0.0,1.3,1,0,0,1,1,1,0,terminase large subunit,MW749003
15020,MK448731_00031,-,Terminase_6N,PF03237.20,6.200000e-01,16.6,4.2,5,13.7,4.1,2.3,1,1,0,1,1,1,0,terminase large subunit,MK448731


9 isn't bad... Let's bias our terminase sample such that we select max 20 from a given family.

In [5]:
tsv_file = "../11Sep2024_vConTACT2_family_annotations.tsv"
annotation_df = pd.read_csv(tsv_file, sep='\t')

# Filter the TSV DataFrame based on the extracted hits
family_df = annotation_df[annotation_df['Node_ID'].isin(filtered_df['genome'])]

In [6]:
family_df.groupby('Family').count()

,Node_ID,Description,Colour (Family)
Family,,,
Ackermannviridae,210,210,210
Aggregaviridae,1,1,1
Aliceevansviridae,72,72,72
Anaerodiviridae,2,2,2
Arenbergviridae,10,10,10
Assiduviridae,4,4,4
Autographiviridae,1628,1628,1628
Chaseviridae,31,31,31
Demerecviridae,350,350,350


In [7]:
classified_df = family_df.loc[family_df['Family'] != 'Unclassified']
classified_df

,Node_ID,Description,Family,Colour (Family)
0,AY319521,Salmonella phage SopEPhi,Peduoviridae,#320B0F
3,MF417929,Uncultured Caudovirales phage clone 2F_1,Peduoviridae,#320B0F
7,AC171169,Escherichia phage H8,Demerecviridae,#EABE70
10,AP013549,uncultured phage_MedDCM-OCT-S39-C11,Autographiviridae,#26AD98
14,AP013545,uncultured phage MedDCM-OCT-S46-C10,Autographiviridae,#26AD98
...,...,...,...,...
24459,AY264777,Escherichia phage T7,Autographiviridae,#26AD98
24460,AY264776,Escherichia phage T7,Autographiviridae,#26AD98
24461,AY264775,Escherichia phage T7,Autographiviridae,#26AD98
24463,AM265638,Pseudomonas phage LKD16,Autographiviridae,#26AD98


In [8]:
# Set the maximum sequences per family and overall max sequences
max_per_family = 20
overall_max = 200

# Example DataFrame (replace with your actual dataframe)
# filtered_df = pd.DataFrame(...)

# Group the remaining sequences by 'Family' and subsample up to 20 sequences per family
grouped_df = classified_df.groupby('Family', group_keys=False).apply(lambda x: x.sample(n=min(len(x), max_per_family)))

# Now, subsample from the grouped dataframe to ensure the overall limit does not exceed 100
if len(grouped_df) > overall_max:
    final_sample = grouped_df.sample(n=overall_max)
else:
    final_sample = grouped_df

In [9]:
final_sample

,Node_ID,Description,Family,Colour (Family)
3034,MW679025,Eggerthella phage LE1-1,Guelinviridae,#E57F04
2399,LC801235,Pseudomonas phage PP21,Zobellviridae,#4CEBB1
18401,MH937503,Streptococcus phage CHPC1084,Aliceevansviridae,#BCA015
3538,OR283208,Microbacterium phage Danimal,Orlajensenviridae,#FF610B
9144,MT310864,Microbacterium phage Nobel,Orlajensenviridae,#FF610B
...,...,...,...,...
18297,MH892376,Streptococcus phage SW1151,Aliceevansviridae,#BCA015
21548,KX349241,Synechococcus phage S-RIM2,Kyanoviridae,#478C15
22276,KR054031,Pseudomonas phage DL62,Autographiviridae,#26AD98
13288,MW929175,Pseudomonas phage K4,Zobellviridae,#4CEBB1


In [10]:
final_sample.groupby('Family').count()

,Node_ID,Description,Colour (Family)
Family,,,
Ackermannviridae,8,8,8
Aliceevansviridae,11,11,11
Anaerodiviridae,1,1,1
Arenbergviridae,3,3,3
Assiduviridae,2,2,2
Autographiviridae,11,11,11
Chaseviridae,7,7,7
Demerecviridae,9,9,9
Drexlerviridae,3,3,3


In [11]:
final_sample.to_csv("sbias_subsample.csv", index=False)

As a sanity check, we want max 200 samples in our tree.

In [13]:
len(final_sample.Node_ID.unique())

200

In [14]:
final_sample.Node_ID.to_csv('headers.txt', index=False, header=False)

In [15]:
tblout_final = filtered_df.loc[filtered_df.genome.isin(final_sample.Node_ID.unique())]


In [16]:
tblout_final = filtered_df.loc[filtered_df.genome.isin(final_sample.Node_ID.unique()) | filtered_df.genome.isin(required_genomes)]
tblout_final

,target_name,accession,query_name,accession_query,e_value,score,bias,e_value_best_dom,score_best_dom,bias_best_dom,exp,reg,clu,ov,env,dom,rep,inc,description,genome
103,HM032710_00181,-,Terminase_6N,PF03237.20,7.500000e-54,189.6,0.0,1.5e-53,188.6,0.0,1.5,1,0,0,1,1,1,1,terminase large subunit,HM032710
143,HQ665011_00180,-,Terminase_6N,PF03237.20,1.800000e-45,162.2,0.2,3.6e-45,161.2,0.1,1.4,2,0,0,2,2,1,1,terminase large subunit,HQ665011
164,JN593240_00155,-,Terminase_6N,PF03237.20,1.500000e-41,149.4,0.1,3.6e-41,148.1,0.1,1.7,1,0,0,1,1,1,1,terminase large subunit,JN593240
207,KC292027_00099,-,Terminase_6N,PF03237.20,9.700000e-26,97.7,0.1,3.4e-25,95.9,0.1,1.9,2,0,0,2,2,1,1,terminase large subunit,KC292027
313,KJ019097_00132,-,Terminase_6N,PF03237.20,1.200000e-51,182.4,0.0,2.9e-51,181.1,0.1,1.7,2,0,0,2,2,1,1,terminase large subunit,KJ019097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799,KF598865_00038,-,Terminase_6N,PF03237.20,9.400000e-03,22.6,0.0,4.4,13.8,0.0,2.4,2,0,0,2,2,2,0,terminase large subunit,KF598865
13810,KF534715_00051,-,Terminase_6N,PF03237.20,1.000000e-02,22.5,0.0,0.026,21.1,0.0,1.7,1,1,0,1,1,1,1,terminase large subunit,KF534715
13823,MN812688_00048,-,Terminase_6N,PF03237.20,1.000000e-02,22.4,0.0,0.026,21.1,0.0,1.7,1,1,0,1,1,1,0,terminase large subunit,MN812688
14699,MW749003_00001,-,Terminase_6N,PF03237.20,1.200000e-01,18.9,0.0,0.19,18.3,0.0,1.3,1,0,0,1,1,1,0,terminase large subunit,MW749003


And after we add our phage group II genomes back in, we have 209, perfect!

In [17]:
tblout_final.target_name.to_csv('proteins_to_pull.txt', index=False, header=False)

Grab these proteins from the body of all Millard proteins.

In [18]:
!esl-sfetch -f "/n/eddy_lab/users/lmerk/millard_sept/inphared_11Sep2024/11Sep2024_vConTACT2_proteins.faa" "proteins_to_pull.txt" > "Terminase_6N.faa"

In case we need a copy of the .faa that doesn't have the "terminase lsu" in the fasta header:

In [19]:
# Input and output file paths
input_faa = "Terminase_6N.faa"  # Replace with your actual .faa file path
output_faa = "Terminase_6N_single_name.faa"

# Function to rename headers
def rename_headers(fasta_file, output_file):
    with open(fasta_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            if line.startswith(">"):
                # Extract only the part before the first underscore
                new_header = re.match(r">(.*?)_", line).group(1)
                outfile.write(f">{new_header}\n")
            else:
                outfile.write(line)

# Run the renaming function
rename_headers(input_faa, output_faa)

print(f"Headers renamed and saved to {output_faa}")

Headers renamed and saved to Terminase_6N_single_name.faa
